In [2]:
import numpy as np
from numpy import dot, floor, argsort
from numpy.random import random, uniform
import numba as nb
import pandas as pd
import time

# 导入数据集
data = np.loadtxt('corel', usecols=range(1, 33))
# 加载预先算好的前1000个点的真实10近邻
true_idxes = pd.read_csv('true_idxes.csv', header=None)
print(data.shape)

(68040, 32)


&emsp;&emsp;欧氏距离对应的LSH Hash Function为：$H(v) =  \lceil\frac{v·r + b}{a}\rceil$，$r$是一个随机向量，$a$是桶宽，$b$是一个在$[0,a]$之间均匀分布的随机变量。$v·r$可以看做是将$v$向$r$上进行投影操作。$H(v)$是(a/2,2a,1/2,1/3)-sensitive的。  

In [2]:
# 计算欧氏距离
# 不用numba加速，用范数计算快，但用了numba，普通计算更快
@nb.jit(nopython=True)
def calc_dist(x, y):
    return np.sqrt(np.sum((x - y) ** 2)) 
    # return np.linalg.norm(x - y)

In [3]:
# R: 32x桶数, b: 1x桶数, a: 标量
# 将向量映射到各桶的索引
def hash_and_fill(inputs, R, b, a):
    # 初始化空的hash_table
    buckets = [{} for _ in range(bucket_num)]
    mapped_idxes = floor((dot(inputs, R) + b) / a) # 68040x10，每一行是这个点在所有桶中的哈希值
    for i, hash_keys in enumerate(mapped_idxes):
        for j, hash_key in enumerate(hash_keys):
            # 每个桶是一个字典，其中的所有key对应该桶的所有索引键值，每个key对应的value是一个list，里面存放映射到该桶、该索引键值的所有点在原数据集的idx
            buckets[j].setdefault(hash_key, []).append(i)
    return buckets  # [ {8: [0, 2, 5,...], 12: [2, 12, 16,...] }, {}, {}, ...]

##### 并集法

In [4]:
# 查询与q相似的向量，并输出相似度最高的k个向量在原数据集中的索引
def find(q, k, R, b, a, buckets):
    hash_keys = np.floor((dot(q, R) + b) / a)[0]   # 不加[0]返回的是1xtables_num的矩阵，取[0]转为数组
    # 遍历q点的索引键列表，找各桶中与其索引键值相等的点
    for i, hash_key in enumerate(hash_keys):
        if i == 0:
            candi_set = set(buckets[0][hash_key])
        else:
            candi_set = candi_set.union(buckets[i][hash_key])  # 候选集：在各桶中的索引与q相同的点取并集（若取交集，会导致候选集元素个数接近1，不现实）
    candi_set = list(candi_set)    # 转为list便于遍历，因为'set' object does not support indexing
    # 遍历候选集，求出离q最近的k个点并返回
    dist = [calc_dist(data[i], q) for i in candi_set]
    set_idxes = argsort(dist)[1: k + 1]  # set_idxes是近邻点在候选集中的索引，要将其转为在原数据集中的索引
    res = [candi_set[i] for i in set_idxes]
    return res

##### 交集法

In [8]:
# 查询与q相似的向量，并输出相似度最高的k个向量在原数据集中的索引
def find2(q, k, R, b, a, buckets):
    hash_val = np.floor((dot(q, R) + b) / a)[0]   # 不加[0]返回的是1xtables_num的矩阵，取[0]转为数组
    # 遍历q点的索引键列表，找各桶中与其索引键值相等的点
    for i, idx_key in enumerate(hash_val):
        if i == 0:
            candi_set = set(buckets[0][idx_key])
        else:
            candi_set = candi_set.intersection(buckets[i][idx_key])  # 候选集：在各桶中的索引与q相同的点取并集（若取交集，会导致候选集元素个数接近1，不现实）
    candi_set = list(candi_set)    # 转为list便于遍历，因为'set' object does not support indexing
    # 遍历候选集，求出离q最近的k个点并返回
    dist = [calc_dist(data[i], q) for i in candi_set]
    set_idxes = argsort(dist)[1: k + 1]  # set_idxes是近邻点在候选集中的索引，要将其转为在原数据集中的索引
    res = [candi_set[i] for i in set_idxes]
    return res

##### 测试一下单点的检索效率

In [6]:
# tables_num: hash_table的个数
# d: 向量的维数
# a是桶宽。a越大，被纳入同个位置的向量就越多，即可以提高原来相似的向量映射到同个位置的概率，反之，则可以降低原来不相似的向量映射到同个位置的概率。因此a需要权衡
bucket_num = 15   # 3
R = random([32, bucket_num])
a = 0.05   # 0.05
b = uniform(0, a, [1, bucket_num])


p = 0
# 进行哈希映射，求出各桶中各索引值下有哪些点
buckets = hash_and_fill(data, R, b, a)

tic = time.clock()
res = find(data[p], 10, R, b, a, buckets)
toc = time.clock()
print('耗时：{:.4f}秒'.format(toc - tic))

# 与暴搜的ground_truth对比
print(res)
dist_real = [calc_dist(data[i], data[p]) for i in range(data.shape[0])]
print(argsort(dist_real)[1 : 11].tolist())

耗时：0.0591秒
[33344, 64186, 51051, 38838, 46277, 61134, 56634, 52696, 64170, 38844]
[33344, 64186, 51051, 38838, 46277, 61134, 56634, 52696, 64170, 38844]


##### 求前1000点的10近邻

In [7]:
# LSH
pred_idx = []
search_lens = []
tic = time.clock()
for i in range(1000):
    res = find(data[i], 10, R, b, a, buckets)
    pred_idx.append(res)
toc = time.clock()
print('耗时：{:.4f}秒'.format(toc - tic))

耗时：68.0579秒


In [ ]:
# 一些实验结果记录：
# a = 0.05, m = 5,  30s,  95.50%
# a = 0.03, m = 5,  40s,  90.11%
# a = 0.01, m = 5,  10s,  50.96%

# a = 0.05, m = 5,  30s,  95.50%
# a = 0.05, m = 10, 53s,  99.74%
# a = 0.05, m = 15, 69s,  99.98%

# a = 0.05, m = 3,  24s,  88.33%
# a = 0.07, m = 3,  36s,  95.10%

##### 计算前1000点的检索precision, recall, accuracy

In [8]:
# 计算准确率
def count_true_num(pred, true):
    a = [i in true for i in pred]
    return sum(a)

n = 1000
precisions = []
recalls = []
accuracys = []
for i in range(n):
    TP = count_true_num(pred_idx[i], true_idxes.iloc[i,:].to_list())
    FP = 10 - TP
    FN = 10 - TP
    TN = 68040 - 10 - FP
    precisions.append(TP / (TP + FP))
    recalls.append(TP / (TP + FN))
    accuracys.append((TP + TN) / (TP + TN + FP + FN))

p, r, acc = np.mean(precisions), np.mean(recalls), np.mean(accuracys)
print("查准率：{:.4f}%\n召回率：{:.4f}%\n准确率：{:.4f}%".format(p*100, r*100, acc*100))

查准率：99.9700%
召回率：99.9700%
准确率：100.0000%


##### 下面是预先算好true_label，存入csv文件，方便求p, r, acc

In [ ]:
# 预先算好true_label，存入csv文件
tic = time.clock()
true_idxes = []
dists = []
for i in range(1000):
    dist_real = [calc_dist(ColorHist[j], ColorHist[i]) for j in range(ColorHist.shape[0])]
    true_idx = np.argsort(dist_real)[1 : 11].tolist()
    true_idxes.append(true_idx)
    dists.append(dist_real)
toc = time.clock()
print('耗时：{:.4f}秒'.format(toc - tic))    # 497s
df1 = pd.DataFrame(true_idxes)
df2 = pd.DataFrame(dists)
df1.to_csv('true_idxes.csv', index = False, header = False)
df2.to_csv('dists.csv', index = False, header = False)